In [25]:
import pandas as pd
import numpy as np

import sys, datetime
sys.path.insert(1, '../../scripts/')
from s3_support import *
%matplotlib inline

### transactions

In [2]:
# data loading
q = '''select
            form,
            recurring,
            payment_type,
            source,
            count(id) as installments,
            sum(amount) as total,
            sum(amount) / count(id) as single_amount,
            max(date) - min(date) as duration,
            min(date) as start_date,
            max(date) as end_date
        from transactions
        where recurring!=0 and status='A'
        group by form, recurring, payment_type, source;'''
rec = redshift_query_read(q, schema='production')

In [3]:
print("Entries: {:,}".format(len(rec)))
print("Recurring: {:,}".format(len(rec['recurring'].unique())))

Entries: 319,002
Recurring: 314,300


### embeds

In [4]:
# embeds
q = "select widget as form from embed"
embeds = redshift_query_read(q, schema='production')

### form upgrades

In [5]:
# logged upgrades
q = "select * from syslog_logs where message like '%Qgiv Form Template Upgraded%'"
form_upgrades = redshift_query_read(q, schema="production")[['org', 'form', 'created']]
forms_upgrades_ids_logged = form_upgrades['form'].unique().tolist()

In [6]:
# created new template forms
bucket = "qgiv-stats-data"
new_form_template_list = "form_download.csv"
new_forms = get_dataframe_from_file(bucket, new_form_template_list)

new_forms['Go Live Date'] = pd.to_datetime(new_forms['Go Live Date'])

new_forms = new_forms[(new_forms['Status']=='active')]
new_forms_ids = new_forms['Form ID'].unique().tolist()

In [7]:
def is_new_template(r):
    try:
        # check for log conversion
        if r['form'] in form_upgrades['form'].unique().tolist():
            # is date after logged upgrade date?
            return r['start_date'] >= form_upgrades[form_upgrades['form']==r['form']]['created'].iloc[0]
        else:
            # form created as new form template
            return r['form'] in new_forms_ids
    except:
        print("error with form {}".format(r['form']))
        raise Exception

### new form settings

In [8]:
q = '''select 
            date, form, appearance, cta_before, 
            cta_after, conditional_fields, image_size
        from analyticsqgiv_weekly
        where 
            date_part('year', date) = 2021 or
            date_part('year', date) = 2022
        order by date asc;'''
nuform_extra = redshift_query_read(q, schema="public")
nuform_extra['multistep'] = nuform_extra['appearance']==2

In [171]:
nuform_extra[['cta_after', 'cta_before']].value_counts()

cta_after  cta_before
0          0             1381931
           1              424264
1          1               33496
           0                4843
dtype: int64

In [9]:
nuform_extra.tail(3)

,date,form,appearance,cta_before,cta_after,conditional_fields,image_size,multistep
1844531,2022-02-14,971742,1,0,0,0,1,False
1844532,2022-02-14,977957,1,0,0,0,2,False
1844533,2022-02-14,958686,2,1,0,0,2,True


In [10]:
def get_new_form_settings(r):
    if r['form'] in nuform_extra['form'].tolist():
        these_settings = nuform_extra[(nuform_extra['form']==r['form'])&(r['start_date']>=nuform_extra['date'])&(r['start_date']<=nuform_extra['date'] + datetime.timedelta(days=7))]
        
        if len(these_settings) > 0:
            return these_settings[['appearance', 'cta_before', 'cta_after', 'conditional_fields', 'multistep', 'image_size']].iloc[0]
    return pd.Series(data={'appearance': None, 'cta_before': None, 'cta_after': None, 'conditional_fields': None, 'multistep': None, 'image_size': None})

### merging

In [11]:
rec['is new template'] = rec[['form', 'start_date']].apply(is_new_template, axis=1)
rec['is embed'] = rec['form'].isin(embeds['form'].tolist())

In [12]:
rec.tail(3)

,form,recurring,payment_type,source,installments,total,single_amount,duration,start_date,end_date,is new template,is embed
318999,693317,1206704,VS,vt,1,10.0,10.0,0 days,2022-02-08,2022-02-08,False,True
319000,951863,1209036,MC,don_form,1,500.0,500.0,0 days,2022-02-23,2022-02-23,False,True
319001,749636,1020019,VS,don_form,1,20.0,20.0,0 days,2022-03-06,2022-03-06,True,True


In [13]:
# new form settings
nuform_settings_df = rec[['form', 'start_date']].apply(get_new_form_settings, axis=1)

KeyboardInterrupt: 

In [ ]:
#rec = rec.merge(nuform_settings_df, how='outer', left_index=True, right_index=True)
rec = rec.merge(nuform_settings_df, left_index=True, right_index=True)

In [ ]:
if 0 in rec.columns:
    rec.drop(0, axis=1, inplace=True)
rec['multistep'] = rec['multistep'].fillna(False)
rec['conditional_fields'] = rec['conditional_fields'].fillna(0)
rec['cta_before'] = rec['cta_before'].fillna(False)
rec['cta_after'] = rec['cta_after'].fillna(False)
rec['image_size'] = rec['image_size'].fillna(False)

In [ ]:
rec.tail(3)

In [ ]:
rec.to_csv("rec.csv", index=False)

In [78]:
rec = pd.read_csv("rec.csv")

/home/ubuntu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (13,14,17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### traffic

In [15]:
q = '''select
            form,
            date,
            sum(views) as views,
            sum(bounces) as bounces
        from googleanalytics_traffic
        group by form, date'''
traffic = redshift_query_read(q, schema='production')

In [16]:
traffic.tail(3)

,form,date,views,bounces
4632665,957295,2022-01-31,1,1
4632666,930910,2022-02-03,1,1
4632667,915672,2022-02-03,2,2


### aggregating

In [80]:
rec['cta_before'].unique()

array([0, 1])

In [79]:
rec['start_date'] = pd.to_datetime(rec['start_date'])
rec['end_date'] = pd.to_datetime(rec['end_date'])
rec['duration'] = pd.to_timedelta(rec['duration'])
rec['cta_before'] = rec['cta_before'].apply(lambda x: 0 if x == 'False' or x is False else 1)
rec['cta_after'] = rec['cta_after'].apply(lambda x: 0 if x == 'False' or x is False else 1)
rec['image_size'] = rec['image_size'].apply(lambda x: np.nan if x == 'False' or x is False else float(x))

In [29]:
rec.head(3)

,form,recurring,payment_type,source,installments,total,single_amount,duration,start_date,end_date,is new template,is embed,appearance,cta_before,cta_after,conditional_fields,multistep,image_size
0,33,2035,MC,don_form,7,70.0,10.0,184 days,2009-05-20,2009-11-20,False,False,NaN,0,0,0.0,False,False
1,217,1258,AM,don_form,18,900.0,50.0,516 days,2009-01-06,2010-06-06,False,False,NaN,0,0,0.0,False,False
2,59,4943,VS,don_form,30,540.0,18.0,203 days,2010-04-29,2010-11-18,False,False,NaN,0,0,0.0,False,False


In [81]:
rec_agg = rec.groupby(['form', 'start_date']).agg({
    'recurring': 'count',
    'installments': 'mean',
    'total': ['sum', 'mean'],
    'single_amount': ['sum', 'mean'],
    'duration': 'mean',
    'is new template': 'mean',
    'is embed': 'mean',
    'appearance': 'mean',
    'cta_before': 'mean',
    'cta_after': 'mean',
    'conditional_fields': 'mean',
    'multistep': 'mean',
    'image_size': 'mean'
}).reset_index()

rec_agg.columns = ['form', 'date', 'recurring', 'installments_mean',
       'total_sum', 'total_mean', 'single_amount_sum',
       'single_amount_mean', 'duration', 'is new template',
       'is embedded', 'appearance', 'cta_before',
       'cta_after', 'conditional_fields', 'multistep',
        'image_size']

In [82]:
rec_agg.tail(3)

,form,date,recurring,installments_mean,total_sum,total_mean,single_amount_sum,single_amount_mean,duration,is new template,is embedded,appearance,cta_before,cta_after,conditional_fields,multistep,image_size
191002,984288,2022-02-18,1,2.0,30.0,30.000,15.00,15.0000,0 days,0.0,1.0,NaN,0.0,0.0,0.0,0.0,NaN
191003,984297,2022-02-18,4,2.0,278.1,69.525,139.05,34.7625,0 days,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN
191004,984297,2022-02-19,2,2.0,123.6,61.800,61.80,30.9000,0 days,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN


### merging

In [83]:
q = '''select
            form,
            date,
            count(id) as onetime,
            sum(amount) as onetime_total,
            avg(amount) as onetime_avg
        from transactions
        where recurring=0 and status='A' 
        group by form, date'''
onetime = redshift_query_read(q, schema='production')

In [84]:
agg = rec_agg.merge(traffic, on=['form', 'date']).merge(onetime, on=['form', 'date'])
agg.tail(3)

,form,date,recurring,installments_mean,total_sum,total_mean,single_amount_sum,single_amount_mean,duration,is new template,...,cta_before,cta_after,conditional_fields,multistep,image_size,views,bounces,onetime,onetime_total,onetime_avg
57299,983349,2022-01-28,5,2.0,267.8,53.56,133.9,26.78,0 days,0.0,...,1.0,1.0,0.0,0.0,2.0,67,46,21,3489.85,166.183333
57300,983349,2022-01-30,1,2.0,61.8,61.80,30.9,30.90,0 days,0.0,...,1.0,1.0,0.0,0.0,2.0,13,9,3,358.00,119.333333
57301,983494,2022-02-02,1,2.0,309.0,309.00,154.5,154.50,0 days,0.0,...,1.0,1.0,0.0,1.0,2.0,8,1,2,250.00,125.000000


In [85]:
agg['transactions'] = agg['recurring'] + agg['onetime']
agg = agg[agg['transactions']<=agg['views']]

agg['conversion'] = agg['transactions'] / agg['views']
agg['conversion_recurring'] = agg['recurring'] / agg['views']
agg['recurring_trans_perc'] = agg['recurring'] / agg['transactions']

### form types & templates

In [86]:
q = "select * from form"
forms = redshift_query_read(q, schema='production')

In [87]:
forms['form_type'] = forms['type']
forms['form_template'] = forms['template']
forms['form'] = forms['id']

agg = agg.merge(forms[['form', 'form_type', 'form_template']], on='form')

# Processing

## 0. Data

In [88]:
rec['year'] = rec['start_date'].dt.year
rec.groupby('year')['recurring'].count()

year
2006        8
2007       90
2008      530
2009     2477
2010     3263
2011     5166
2012     7907
2013    10615
2014    15778
2015    18258
2016    20593
2017    25893
2018    35359
2019    39811
2020    55667
2021    66466
2022     8613
Name: recurring, dtype: int64

In [89]:
agg['year'] = agg['date'].dt.year
agg.groupby('year')[['recurring', 'onetime']].sum().reset_index()

,year,recurring,onetime
0,2016,1086,7071
1,2017,11775,56594
2,2018,13445,61053
3,2019,14955,71126
4,2020,18738,146069
5,2021,21721,136941
6,2022,2209,13819


## 1. Recurring donations processed on legacy vs new forms

- number of recurring on forms with nudges
- number of recurring on forms with modals
- number of recurring on forms with both nudges and modals
- number of recurring on forms with neither nudges nor modals

In [90]:
agg.groupby('cta_before')[['conversion_recurring', 'recurring_trans_perc', 'installments_mean', 'total_sum', 'total_mean', 'single_amount_sum', 'single_amount_mean']].mean().transpose()

cta_before,0.0,1.0
conversion_recurring,0.078361,0.079680
recurring_trans_perc,0.304049,0.265347
installments_mean,13.981372,6.076575
total_sum,1457.057805,734.050026
total_mean,1036.295439,435.352791
single_amount_sum,217.681917,177.778484
single_amount_mean,137.125098,103.732662


In [91]:
agg.groupby('cta_after')[['conversion_recurring', 'recurring_trans_perc', 'installments_mean', 'total_sum', 'total_mean', 'single_amount_sum', 'single_amount_mean']].mean().transpose()

cta_after,0.0,1.0
conversion_recurring,0.078361,0.079680
recurring_trans_perc,0.304049,0.265347
installments_mean,13.981372,6.076575
total_sum,1457.057805,734.050026
total_mean,1036.295439,435.352791
single_amount_sum,217.681917,177.778484
single_amount_mean,137.125098,103.732662


In [92]:
print("both before & after")
agg[(agg['cta_before']>0)&(agg['cta_after']>0)][['conversion_recurring', 'recurring_trans_perc', 'installments_mean', 'total_sum', 'total_mean', 'single_amount_sum', 'single_amount_mean']].mean().transpose()

both before & after


conversion_recurring      0.079680
recurring_trans_perc      0.265347
installments_mean         6.076575
total_sum               734.050026
total_mean              435.352791
single_amount_sum       177.778484
single_amount_mean      103.732662
dtype: float64

In [93]:
print("neither before nor after")
agg[(agg['cta_before']==0)&(agg['cta_after']==0)][['conversion_recurring', 'recurring_trans_perc', 'installments_mean', 'total_sum', 'total_mean', 'single_amount_sum', 'single_amount_mean']].mean().transpose()

neither before nor after


conversion_recurring       0.078361
recurring_trans_perc       0.304049
installments_mean         13.981372
total_sum               1457.057805
total_mean              1036.295439
single_amount_sum        217.681917
single_amount_mean       137.125098
dtype: float64

In [169]:
agg[['cta_before', 'cta_after']].value_counts()

cta_before  cta_after
0.0         0.0          42601
1.0         1.0           9927
dtype: int64

## 2. Recurring single step vs multi-step

- number of recurring on single-step forms & conversion rate by number of fields
- number of recurring on multi-step forms & conversion rate by number of fields

In [94]:
agg.groupby('multistep')[['conversion_recurring', 'recurring_trans_perc', 'installments_mean', 'total_sum', 'total_mean', 'single_amount_sum', 'single_amount_mean']].mean().transpose()

multistep,0.0,1.0
conversion_recurring,0.077033,0.104164
recurring_trans_perc,0.296435,0.301598
installments_mean,12.946666,5.048863
total_sum,1353.850970,778.851107
total_mean,946.577131,536.349772
single_amount_sum,207.768854,248.565196
single_amount_mean,128.266833,172.084857


## 3. Recurring using amounts w/ images

- number of recurring on forms with amounts w/ images
- number of recurring on forms with amounts w/out images
- number of recurring on forms w/ no amounts

In [95]:
agg.groupby('image_size')[['conversion_recurring', 'recurring_trans_perc', 'installments_mean', 'total_sum', 'total_mean', 'single_amount_sum', 'single_amount_mean']].mean().transpose()

image_size,0.0,1.0,2.0
conversion_recurring,0.076143,0.067852,0.093636
recurring_trans_perc,0.285612,0.243644,0.284434
installments_mean,10.095311,6.176412,4.986825
total_sum,1036.633158,613.010160,794.316029
total_mean,578.302796,352.478318,492.265555
single_amount_sum,132.121265,129.708704,242.118409
single_amount_mean,71.185415,70.398389,148.565444


## 4. Abandonment rate

- one time vs recurring

## 5. Recurring payment methods

- check vs card
- card type
- P2P payment type
- SMS payment type
- VT payment type

In [96]:
print("Check vs Card")
print("-"*40)
chk_len = len(rec[rec['payment_type']=='ck'])
chk_perc = (chk_len / len(rec)) * 100.
crd_len = len(rec[rec['payment_type']!='ck'])
crd_perc = (crd_len / len(rec)) * 100.

print("Check: {:,} ({:.2f}%)".format(chk_len, chk_perc))
print("Card: {:,} ({:.2f}%)".format(crd_len, crd_perc))

Check vs Card
----------------------------------------
Check: 20,909 (6.61%)
Card: 295,585 (93.39%)


In [97]:
print("Card type")
print("-"*40)
print(rec['payment_type'].value_counts())
print(rec['payment_type'].value_counts(normalize=True) * 100.)

Card type
----------------------------------------
VS    191501
MC     68666
AM     21245
ck     20909
DC      9770
PP      4362
"          2
no         1
Name: payment_type, dtype: int64
VS    60.514258
MC    21.698435
AM     6.713414
ck     6.607238
DC     3.087317
PP     1.378391
"      0.000632
no     0.000316
Name: payment_type, dtype: float64


In [98]:
print("P2P card type")
print("-"*40)
print(rec[rec['source']=='p2p']['payment_type'].value_counts())
print(rec[rec['source']=='p2p']['payment_type'].value_counts(normalize=True) * 100.)

P2P card type
----------------------------------------
VS    3053
MC     979
AM     446
DC     168
ck      56
PP       4
Name: payment_type, dtype: int64
VS    64.874628
MC    20.803230
AM     9.477263
DC     3.569911
ck     1.189970
PP     0.084998
Name: payment_type, dtype: float64


In [99]:
print("SMS card type")
print("-"*40)
print(rec[rec['source']=='sms']['payment_type'].value_counts())
print(rec[rec['source']=='sms']['payment_type'].value_counts(normalize=True) * 100.)

SMS card type
----------------------------------------
VS    749
MC    234
AM     54
ck     40
DC     23
Name: payment_type, dtype: int64
VS    68.090909
MC    21.272727
AM     4.909091
ck     3.636364
DC     2.090909
Name: payment_type, dtype: float64


In [100]:
print("VT card type")
print("-"*40)
print(rec[rec['source']=='vt']['payment_type'].value_counts())
print(rec[rec['source']=='vt']['payment_type'].value_counts(normalize=True) * 100.)

VT card type
----------------------------------------
VS    19241
MC     6468
ck     3491
AM     2343
DC      952
Name: payment_type, dtype: int64
VS    59.212186
MC    19.904601
ck    10.743191
AM     7.210340
DC     2.929681
Name: payment_type, dtype: float64


## 6. Recurring embedded vs landing pages

In [101]:
agg.groupby('is embedded')[['conversion_recurring', 'recurring_trans_perc', 'installments_mean', 'total_sum', 'total_mean', 'single_amount_sum', 'single_amount_mean']].mean().transpose()

is embedded,0.0,1.0
conversion_recurring,0.069787,0.084437
recurring_trans_perc,0.273183,0.312288
installments_mean,11.408858,13.199796
total_sum,1160.406133,1426.091454
total_mean,777.669649,1018.519878
single_amount_sum,166.529887,238.940826
single_amount_mean,109.961794,144.585239


In [102]:
agg['form_type'].value_counts()

1    52374
3       97
2       57
Name: form_type, dtype: int64

In [103]:
agg[agg['form_type']==4][['conversion_recurring', 'recurring_trans_perc', 'installments_mean', 'total_sum', 'total_mean', 'single_amount_sum', 'single_amount_mean']].mean().transpose()

conversion_recurring   NaN
recurring_trans_perc   NaN
installments_mean      NaN
total_sum              NaN
total_mean             NaN
single_amount_sum      NaN
single_amount_mean     NaN
dtype: float64

## 7. Recurring billing patterns

- % forms w/ start/end dates enabled
- % forms w/out start/end dates enabled
- performance forms w/ start/end dates enabled vs not allowed
- % recurring set up w/ first payment in the future
- % recurring set up w/ first payment immediately
- % recurring set up w/ end date

# Recurring Donor Behavior

## 1. General recurring donation data points
- ~Average one-time gift size~
- ~Average recurring gift size~
    - Goal: compare average gift sizes for one-time and recurring gifts
- ~Average number of installments for recurring gifts~

_Covered in \#1 of prior section_

## 2. Recurring donor lifetime information
- % of recurring gifts greater than 1 year
- % of recurring gifts less than 1 year
- Average length of recurring gift
- Goal: understand the donor lifetime value of recurring donors

In [104]:
len_gt_year = len(rec[rec['duration']>datetime.timedelta(days=365)])
perc_gt_year = (len_gt_year / len(rec)) * 100.

print("% greater than year: {:.2f}%".format(perc_gt_year))

print("Mean duration: {}".format(rec['duration'].mean()))
print("Median duration: {}".format(rec['duration'].median()))

% greater than year: 30.21%
Mean duration: 349 days 14:47:13.782630968
Median duration: 215 days 00:00:00


## 3. Recurring donor retention
- Is this possible to see in the Qgiv system?
- % of recurring donations canceled after first installment
- % of recurring donations paused by the system
- % of recurring donations restarted after a pause
    - \# un-paused by donors
    - \# un-paused by nonprofit

In [105]:
len_installments_1 = len(rec[rec['installments']==1])

perc_installments_1 = (len_installments_1 / len(rec)) * 100.

print("% installments 1: {:.2f}%".format(perc_installments_1))

% installments 1: 8.00%


In [106]:
# paused recurring
q = "select created, org, form, userid, entity, entitytype, message from logs where message like '%paused recurring%'"
rec_paused = redshift_query_read(q, schema='production')

In [107]:
len_paused = len(rec_paused)
len_paused_unique = len(rec_paused['message'].unique())
perc_paused = (len_paused / len(rec)) * 100.
perc_paused_unique = (len_paused_unique / len(rec)) * 100.

print("paused recurring: {:.2f}%".format(perc_paused))
print("paused recurring unique: {:.2f}%".format(perc_paused_unique))

paused recurring: 5.87%
paused recurring unique: 3.74%


In [108]:
rec_paused['recurring'] = rec_paused['message'].str.replace("paused recurring (", "", regex=False)
rec_paused['recurring'] = rec_paused['recurring'].str.replace(")", '', regex=False)

In [109]:
rec_paused_agg = rec_paused.groupby('recurring')['created'].agg(['max', 'count']).reset_index()
rec_paused_agg.columns = ['recurring', 'paused_last', 'paused_count']
rec_paused_agg['paused_count'].value_counts(normalize=True)[:4]

1    0.547953
2    0.399357
4    0.022243
3    0.021313
Name: paused_count, dtype: float64

In [110]:
rec_paused_agg['recurring'] = rec_paused_agg['recurring'].astype(int)
rec = rec.merge(rec_paused_agg, on='recurring', how='outer')

In [111]:
len(rec), len(rec[rec['form'].isna()])

(316873, 379)

In [112]:
print("dropping NA forms:")
print()

paused_restarted = len(rec[(~rec['form'].isna())&(rec['paused_count']>0)&(rec['paused_last']<rec['end_date'])])
perc_paused_restarted = (paused_restarted / len(rec[~rec['form'].isna()])) * 100.

len_paused = len(rec[(~rec['form'].isna())&(rec['paused_count']>0)])
perc_paused = (len_paused / len(rec[~rec['form'].isna()])) * 100.

print("Paused & restarted: {:.2f}%".format(perc_paused_restarted))
print("Paused: {:.2f}%".format(perc_paused))

dropping NA forms:

Paused & restarted: 0.38%
Paused: 3.73%


## 4. Donor accounts
- % of recurring donations tied to a donor account
- % of recurring donations not tied to a donor account
- % of recurring donations updated through donor accounts
    - Updated frequency
    - Updated billing or end dates
    - Updated amounts
    - Updated payment information

## 5. Recurring gift creation by time
- What part of the month do donors create recurring gifts?
- What days of the week do donors create recurring gifts?
- What time of day do donors create recurring gifts?
- Goal: understand when donors are most likely to create recurring gifts
    - Would be great to help nonprofits time their appeals for ongoing donations
    - Idea: create a graph showing when donors are most and least likely to set up a recurring gift

In [113]:
rec['start_dom'] = rec['start_date'].dt.day
rec['start_dow'] = rec['start_date'].dt.dayofweek

In [114]:
rec['start_dow'].value_counts(normalize=True)

1.0    0.156297
4.0    0.153561
3.0    0.146897
0.0    0.146714
2.0    0.141131
6.0    0.129102
5.0    0.126299
Name: start_dow, dtype: float64

In [115]:
rec_dom = rec['start_dom'].value_counts(normalize=True)
rec_dom.head(10)

1.0     0.080886
15.0    0.060346
5.0     0.039770
2.0     0.035539
30.0    0.035223
3.0     0.035075
20.0    0.034753
10.0    0.031912
16.0    0.031896
4.0     0.031179
Name: start_dom, dtype: float64

In [116]:
print("29 - 2: {:.2f}%".format(rec_dom[[1.0, 2.0, 29.0, 30.0, 31.0]].sum() * 100.))
print("14 - 16: {:.2f}%".format(rec_dom[[14.0, 15.0, 16.0]].sum() * 100.))

29 - 2: 20.28%
14 - 16: 11.87%


## 6. Recurring donations to organizations by mission

- segment
- pricing package for fun

In [117]:
org_export = get_dataframe_from_file('qgiv-stats-data', 'org_download.csv')

In [118]:
org_export = org_export[['Id', 'Segment', 'Pricing Package']]
org_export.columns = ['org', 'segment', 'pricing_package']
org_export.head(3)

,org,segment,pricing_package
0,441789,Z - Unknown,Start
1,443044,NaN,Start
2,1045,NaN,Legacy Pricing


In [119]:
q = "select id as form, org from form"
form_orgs = redshift_query_read(q, schema='production')

In [120]:
print("merging for org ID")
print("original agg len: {:,}".format(len(agg)))
agg_mrgd = agg.merge(form_orgs, on='form')
print("merged agg len: {:,}".format(len(agg_mrgd)))

merging for org ID
original agg len: 52,528
merged agg len: 52,528


In [121]:
print("merging for org segment & pricing package")
print("original agg len: {:,}".format(len(agg)))
agg_mrgd = agg_mrgd.merge(org_export, on='org')
print("merged agg len: {:,}".format(len(agg_mrgd)))

merging for org segment & pricing package
original agg len: 52,528
merged agg len: 52,528


### org segment

In [122]:
agg_mrgd.groupby('segment')[['conversion_recurring', 'recurring_trans_perc', 'installments_mean', 'total_sum', 'total_mean', 'single_amount_sum', 'single_amount_mean']].mean().reset_index()

,segment,conversion_recurring,recurring_trans_perc,installments_mean,total_sum,total_mean,single_amount_sum,single_amount_mean
0,"A - Arts, Culture, and Humanities",0.127938,0.376660,8.875017,665.280487,562.447823,137.157418,119.946657
1,B - Educational Institutions,0.095436,0.343739,8.212926,2217.981190,1325.665936,1173.351094,568.236112
2,C - Environmental Advocacy and Protection,0.136583,0.401632,7.066220,378.065977,292.294627,108.726879,78.767689
3,D - Animal,0.078362,0.331604,12.525801,598.991869,395.630630,61.189407,39.249649
4,"E - Health, General and Rehabilitative",0.103059,0.368733,11.671027,913.306431,691.598980,124.028590,103.397637
5,F - Mental Health and Crisis Intervention,0.109589,0.380259,13.892832,1382.867020,1039.480320,132.657565,102.913344
6,"G - Disease, Disorders, Medical Disciplines",0.101182,0.318549,9.810828,700.281802,594.853247,118.460454,101.585929
7,H - Medical Research,0.065081,0.287548,11.692959,918.874347,701.732940,104.995809,88.015317
8,"I - Crime, Legal Related",0.100192,0.364521,10.418519,998.645812,826.828036,188.130567,151.173269
9,"J - Employment, Job Related",0.074888,0.367535,11.891156,618.618163,574.366463,133.849796,127.914422


### pricing package

In [123]:
agg_mrgd.groupby('pricing_package')[['conversion_recurring', 'recurring_trans_perc', 'installments_mean', 'total_sum', 'total_mean', 'single_amount_sum', 'single_amount_mean']].mean().reset_index()

,pricing_package,conversion_recurring,recurring_trans_perc,installments_mean,total_sum,total_mean,single_amount_sum,single_amount_mean
0,Grow / Mobile Suite,0.089692,0.275795,9.633157,848.802434,718.355758,91.154869,77.553738
1,Legacy Pricing,0.085717,0.300344,13.747262,2233.518831,1907.230401,219.822768,186.279052
2,Package Pricing,0.094619,0.326753,12.221069,1406.563343,1073.736709,189.520317,148.001474
3,Package Pricing - Auction Package,0.071090,0.310357,16.218497,1615.602955,1411.544179,154.345815,141.263490
4,"Package Pricing - Auction Package, Data Package",0.141781,0.444800,5.677273,530.598636,465.946760,124.734595,112.157977
5,"Package Pricing - Auction Package, Data Packag...",0.104493,0.425168,10.813853,1095.703636,850.405887,351.514378,317.751550
6,"Package Pricing - Auction Package, Data Packag...",0.034153,0.354167,5.750000,551.875000,551.875000,90.625000,90.625000
7,"Package Pricing - Auction Package, Peer-to-Pee...",0.074228,0.370563,10.181818,622.892727,408.347273,61.024848,49.509697
8,"Package Pricing - Auction Package, Text Fundra...",0.062778,0.249693,12.306623,1191.844570,952.773726,160.540356,132.213285
9,"Package Pricing - Bill To Carrier, Data Packag...",0.079604,0.300000,7.500000,245.100000,245.100000,66.600000,66.600000


## 7. Donor location
- State
- Zip code
- Idea: create a “wealth map” that shows recurring donors’ locations

_This can be presented in a much more intelligible and aesthetically pleasing way in Tableau_

In [124]:
q = '''select
           id, 
           recurring, 
           org, 
           form, 
           amount, 
           state,
           zip
    from transactions
    where recurring!=0 and status='A' '''
rec_loc = redshift_query_read(q, schema='production')
rec_loc = rec_loc.groupby('recurring').first()

In [125]:
len(rec_loc['state'].unique()), print(rec_loc['state'].unique())

['FL' 'NY' 'MI' ... 'Amaz' 'Barn' 'Glac']


(1186, None)

In [126]:
rec_loc['state'].value_counts(normalize=True).head(25)

CA    0.142114
FL    0.091420
TX    0.070934
PA    0.056756
NY    0.051431
CO    0.035687
VA    0.033965
NC    0.030213
WA    0.030137
MD    0.029272
OH    0.027975
IL    0.024846
GA    0.024296
MI    0.022391
AZ    0.020205
OR    0.019856
NJ    0.018307
TN    0.016322
OK    0.015516
MA    0.013763
SC    0.012373
AL    0.012210
MO    0.012072
NV    0.011678
IN    0.011284
Name: state, dtype: float64

In [127]:
rec_loc.groupby('state')['amount'].sum().reset_index().sort_values('amount', ascending=False).head(10)

,state,amount
175,CA,5909185.17
568,MI,3509821.57
310,FL,3107815.68
9,AL,2547258.49
946,TX,2081342.64
754,PA,1160816.05
668,NY,961528.88
183,CO,917248.78
642,NC,812657.16
716,OH,796555.13


## 8. General donor demographics and other fun things
- % of recurring donations made on mobile forms
    - Overall
    - On Android devices
    - On iOS devices
- % of recurring donations made on desktop devices
    - Overall
    - On Windows devices
    - On Apple devices
    - On other devices
    - Do Linux users even exist any more?
- % of recurring donations made via text
    - Text-to-donate
    - Bill to carrier
    - On Android devices
    - On iOS devices
- Breakdown of recurring donations made on different web - browsers
    - Edge
    - Chrome
    - FireFox
    - Opera
    - Safari
    - Whatever other weird browsers people use








In [128]:
q = '''select
           id, 
           recurring, 
           org, 
           form, 
           amount, 
           payment_type, 
           date, 
           hour, 
           useragent,
           source
    from transactions
    where recurring!=0 and status='A' '''
rec_ua = redshift_query_read(q, schema='production')
rec_ua = rec_ua.groupby('recurring').first()

### hour

In [129]:
rec_ua['hour'].value_counts(normalize=True)

7     0.236118
12    0.197725
13    0.092342
8     0.057391
14    0.036118
11    0.034989
23    0.034528
15    0.032943
16    0.030948
10    0.030140
9     0.028113
17    0.027232
6     0.026532
19    0.024149
18    0.022991
20    0.020127
21    0.018155
22    0.015196
2     0.009252
0     0.007916
1     0.006287
5     0.003770
4     0.003627
3     0.003411
Name: hour, dtype: float64

In [130]:
rec_ua.groupby('hour')['amount'].agg(['mean', 'median', 'sum']).reset_index()

,hour,mean,median,sum
0,0,495.821298,25.00,1233603.39
1,1,616.224595,25.00,1217659.80
2,2,78.357074,25.00,227862.37
3,3,64.416073,20.00,69054.03
4,4,74.653465,20.00,85104.95
5,5,91.534768,20.79,108468.70
6,6,82.253683,26.25,685913.46
7,7,73.547079,25.87,5458075.83
8,8,156.896178,30.90,2830093.26
9,9,124.770111,29.00,1102468.70


### user agent

In [131]:
def tag_useragent(r):
    if not r:
        return None
    elif 'iPhone' in r:
        return 'iPhone'
    elif 'iPad' in r:
        return 'iPad'
    elif 'Android' in r:
        return 'Android'
    elif 'Windows' in r:
        return 'Windows'
    elif 'Mac' in r:
        return 'Mac'
    elif 'CrOS' in r:
        return 'ChromeOS'
    elif 'Linuxx8664' in r:
        return 'Linuxx8664'
    return None

In [132]:
rec_ua['platform'] = rec_ua['useragent'].apply(tag_useragent)

In [133]:
rec_ua['platform'].value_counts(normalize=True)

Windows       0.426097
iPhone        0.231084
Mac           0.190140
Android       0.122660
iPad          0.014623
ChromeOS      0.012014
Linuxx8664    0.003383
Name: platform, dtype: float64

In [134]:
rec_ua.groupby('platform')['amount'].agg(['mean', 'median', 'sum']).reset_index()

,platform,mean,median,sum
0,Android,68.226205,25.000,583743.41
1,ChromeOS,62.403974,25.000,52294.53
2,Linuxx8664,84.703771,25.735,19990.09
3,Mac,105.363078,30.000,1397430.50
4,Windows,85.082865,25.000,2528832.90
5,iPad,56.999716,25.000,58139.71
6,iPhone,80.626956,25.420,1299625.91


### source

In [135]:
rec_ua['source'].value_counts(normalize=True)

don_form    0.766564
vt          0.100493
mobile      0.099526
p2p         0.015081
kiosk       0.012704
sms         0.003538
mobilevt    0.000789
givi        0.000700
fb          0.000605
Name: source, dtype: float64

In [136]:
rec_ua.groupby('source')['amount'].agg(['mean', 'median', 'sum']).reset_index()

,source,mean,median,sum
0,don_form,108.122389,26.00,26050035.38
1,fb,44.538053,25.00,8462.23
2,givi,83.625591,60.00,18397.63
3,kiosk,92.226294,50.00,368259.59
4,mobile,166.115980,30.88,5196273.97
5,mobilevt,89.311048,40.00,22149.14
6,p2p,69.528878,25.00,329566.88
7,sms,58.570989,20.00,65130.94
8,vt,111.312427,38.00,3515803.02


## form recurring settings

In [160]:
agg_mrgd['date'].min(), agg_mrgd['date'].max()

(Timestamp('2016-12-08 00:00:00'), Timestamp('2022-02-03 00:00:00'))

In [139]:
q = "select id as form, org, type, template, enableenddate, enablestartdate, recurringplanenddaterequired from form"
form_settings = redshift_query_read(q, schema='production')

In [140]:
form_settings.tail()

,form,org,type,template,enableenddate,enablestartdate,recurringplanenddaterequired
67727,985198,447050,1,8,False,False,False
67728,985214,442273,1,8,False,False,False
67729,985230,444775,1,6,True,False,False
67730,985246,447896,1,8,True,True,False
67731,985262,443030,1,8,False,True,False


In [146]:
agg_mrgd = agg_mrgd.merge(form_settings.drop('org', axis=1), on='form')

In [156]:
agg_mrgd.groupby('enableenddate')[['conversion_recurring', 'recurring_trans_perc', 'installments_mean', 'total_sum', 'total_mean', 'single_amount_sum', 'single_amount_mean', 'duration']].mean().transpose()

enableenddate,False,True
conversion_recurring,0.059898,0.093910
recurring_trans_perc,0.266161,0.321733
installments_mean,13.710920,11.487155
total_sum,1199.198290,1419.536239
total_mean,756.735923,1058.446868
single_amount_sum,117.719181,285.708427
single_amount_mean,69.297299,181.113345


In [157]:
agg_mrgd.groupby('enablestartdate')[['conversion_recurring', 'recurring_trans_perc', 'installments_mean', 'total_sum', 'total_mean', 'single_amount_sum', 'single_amount_mean', 'duration']].mean().transpose()

enablestartdate,False,True
conversion_recurring,0.077504,0.097905
recurring_trans_perc,0.295830,0.312526
installments_mean,12.583435,10.813754
total_sum,1313.395114,1442.965204
total_mean,914.542477,1065.484166
single_amount_sum,205.837216,285.211190
single_amount_mean,126.997621,197.394174


In [158]:
agg_mrgd.groupby('recurringplanenddaterequired')[['conversion_recurring', 'recurring_trans_perc', 'installments_mean', 'total_sum', 'total_mean', 'single_amount_sum', 'single_amount_mean', 'duration']].mean().transpose()

recurringplanenddaterequired,False,True
conversion_recurring,0.078606,0.091232
recurring_trans_perc,0.296715,0.357970
installments_mean,12.488795,8.441176
total_sum,1320.515753,1025.379412
total_mean,922.813005,655.028765
single_amount_sum,210.139340,214.559281
single_amount_mean,130.817105,122.555399


In [153]:
agg_mrgd.groupby('type')[['conversion_recurring', 'recurring_trans_perc', 'installments_mean', 'total_sum', 'total_mean', 'single_amount_sum', 'single_amount_mean']].mean().transpose()

type,1,2,3
conversion_recurring,0.078687,0.119274,0.013358
recurring_trans_perc,0.296814,0.389586,0.199476
installments_mean,12.497913,11.699196,7.319978
total_sum,1321.888967,744.017544,866.107526
total_mean,924.074995,551.395102,412.740800
single_amount_sum,210.415351,76.403509,140.471829
single_amount_mean,130.989719,50.541545,83.340166


In [154]:
agg_mrgd.groupby('template')[['conversion_recurring', 'recurring_trans_perc', 'installments_mean', 'total_sum', 'total_mean', 'single_amount_sum', 'single_amount_mean']].mean().transpose()

template,0,6,7,8,9,10
conversion_recurring,0.050564,0.058803,0.3625,0.096046,0.012371,0.036301
recurring_trans_perc,0.245526,0.255981,0.4250,0.332459,0.196940,0.258433
installments_mean,16.407795,13.215632,6.2500,11.832213,7.416177,5.083333
total_sum,1162.261576,1318.382428,782.0000,1325.977581,840.778817,1455.000000
total_mean,881.508721,878.493220,782.0000,962.738222,388.844347,968.333333
single_amount_sum,94.832305,139.397881,119.0000,272.219128,126.513628,465.000000
single_amount_mean,75.680076,93.221859,119.0000,163.749012,72.372718,338.333333


## Recurring prompts 10/01/2020 - 1/1/2022

In [165]:
agg_mrgd[(agg_mrgd['date']>='2020-10-01')&(agg_mrgd['date']<='2022-01-01')].groupby('cta_before')[['conversion_recurring', 'recurring_trans_perc', 'installments_mean', 'total_sum', 'total_mean', 'single_amount_sum', 'single_amount_mean', 'duration']].mean().transpose()

cta_before,0.0,1.0
conversion_recurring,0.069351,0.080247
recurring_trans_perc,0.251471,0.263872
installments_mean,10.870020,6.493041
total_sum,1001.956183,766.847598
total_mean,586.906166,460.563930
single_amount_sum,124.677306,175.734943
single_amount_mean,77.402447,105.187632


In [166]:
agg_mrgd[(agg_mrgd['date']>='2020-10-01')&(agg_mrgd['date']<='2022-01-01')].groupby('cta_after')[['conversion_recurring', 'recurring_trans_perc', 'installments_mean', 'total_sum', 'total_mean', 'single_amount_sum', 'single_amount_mean', 'duration']].mean().transpose()

cta_after,0.0,1.0
conversion_recurring,0.069351,0.080247
recurring_trans_perc,0.251471,0.263872
installments_mean,10.870020,6.493041
total_sum,1001.956183,766.847598
total_mean,586.906166,460.563930
single_amount_sum,124.677306,175.734943
single_amount_mean,77.402447,105.187632


In [167]:
agg_mrgd[(agg_mrgd['date']>='2020-10-01')&(agg_mrgd['date']<='2022-01-01')].groupby(['cta_after', 'cta_before'])[['conversion_recurring', 'recurring_trans_perc', 'installments_mean', 'total_sum', 'total_mean', 'single_amount_sum', 'single_amount_mean', 'duration']].mean().transpose()

cta_after,0.0,1.0
cta_before,0.0,1.0
conversion_recurring,0.069351,0.080247
recurring_trans_perc,0.251471,0.263872
installments_mean,10.870020,6.493041
total_sum,1001.956183,766.847598
total_mean,586.906166,460.563930
single_amount_sum,124.677306,175.734943
single_amount_mean,77.402447,105.187632


In [168]:
agg_mrgd[['cta_after', 'cta_before']].value_counts()

cta_after  cta_before
0.0        0.0           42601
1.0        1.0            9927
dtype: int64

In [174]:
agg_mrgd[['form', 'date', 'cta_before', 'cta_after']].tail()

,form,date,cta_before,cta_after
52523,982245,2022-01-28,1.0,1.0
52524,982245,2022-01-29,1.0,1.0
52525,982245,2022-01-30,1.0,1.0
52526,982724,2022-01-20,1.0,1.0
52527,982724,2022-01-26,1.0,1.0


# load to redshift

In [184]:
#agg_mrgd.to_csv("recurring_agg.csv", index=False)

In [194]:
# data prep
int_cols = ['form', 'org', 'recurring', 'views',
           'bounces', 'onetime', 'transactions', 
           'form_type', 'form_template', 'type']
for c in int_cols:
    agg_mrgd[c] = agg_mrgd[c].astype(int).fillna(0)
    
# string formatting
agg_mrgd['segment'] = agg_mrgd['segment'].str.replace(',', ';')
agg_mrgd['pricing_package'] = agg_mrgd['pricing_package'].str.replace(',', ';')

agg_mrgd['duration'] = agg_mrgd['duration'].astype(str).str.slice(0, 25)

In [202]:
bool_cols = ['is new template', 'is embedded', 
             'enableenddate', 'enablestartdate',
             'recurringplanenddaterequired']
for c in bool_cols:
    agg_mrgd[c] = agg_mrgd[c].astype(bool)

In [212]:
agg_mrgd['is_new_template'] = agg_mrgd['is new template']
agg_mrgd['is_embedded'] = agg_mrgd['is embedded']

agg_mrgd['appearance'] = agg_mrgd['appearance'].apply(lambda x: x if x else 0).fillna(0)
agg_mrgd['image_size'] = agg_mrgd['image_size'].apply(lambda x: x if x else 0).fillna(0)

In [219]:
cols = ['form', 'org', 'date', 'recurring', 'installments_mean',
        'total_sum', 'total_mean', 'single_amount_sum',
        'single_amount_mean', 'duration', 'is_new_template',
        'is_embedded', 'appearance', 'cta_before', 'cta_after',
        'conditional_fields', 'multistep', 'image_size',
        'views', 'bounces', 'onetime', 'onetime_total',
        'onetime_avg', 'transactions', 'conversion',
        'conversion_recurring', 'recurring_trans_perc',
        'form_type', 'form_template', 'segment',
        'pricing_package', 'type', 'template', 'enableenddate',
        'enablestartdate', 'recurringplanenddaterequired']

save_dataframe_to_file("qgiv-stats-data", "recurring_agg.csv", agg_mrgd[cols])

uploading to S3
Done


In [220]:
q = '''copy recurring_agg
        from 's3://qgiv-stats-data/recurring_agg.csv'
        iam_role 'arn:aws:iam::637885584661:role/AWSRoleForRedshift'
        emptyasnull
        blanksasnull
        fillrecord
        delimiter ','
        ignoreheader 1
        region 'us-east-1'
        '''
#redshift_query_write(q, schema='public')

Call to action before | False | True |
converison               7.83%